# Teaming - IS843


The seed below ensures that the results are random, but reproducible!

In [ ]:
seed = 843

Reading the data (data is kept private for students' privacy)

In [ ]:
students = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load('gs://is843/data/students-is843-spring-2022.csv')

students.show()

Leads were selected in the class:

In [ ]:
from pyspark.sql.functions import *
import pandas as pd

leads = students.where("team").select("name")

print("after shuffling:")
leads = leads.orderBy(rand(seed))

leads.show()

In [ ]:
all_teams = pd.DataFrame()

for i in range(8):
    take_n = 4
    if i < 5: take_n += 1
    group = students.where("!team").select("name").orderBy(rand(seed)).take(take_n)
    print("\n", "Team ", i+1, ": ", sep="")
    group_l = [x[0] for x in group]
    students = students.withColumn("team", when(col("name").isin(group_l), True).otherwise(col("team")))
    print(leads.collect()[i][0])
    print(*group_l, sep = "\n")
    print("\n", "-----------------------------------", sep = "")
    
    group_l.insert(0, leads.collect()[i][0])
    teammates = pd.DataFrame({
        'Name': group_l,
        'Team': i+1})
    all_teams = pd.concat([all_teams, teammates])

In [ ]:
pd.merge(all_teams, students.toPandas()[['Name', 'Email']])[['Name', 'Email', 'Team']].to_csv('teams.csv')